<a href="https://colab.research.google.com/github/Gijslake/DDWT19/blob/master/skills_classificatie/training/Classifier_op_model_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifier op model embeddings

## Preperations

In [1]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U scikit-learn
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 25.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


## Data ophalen

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn import metrics
from ast import literal_eval
import json
import re

# data laden
csv_path = f"/content/drive/MyDrive/Osso data/training data/classifier_data_250_vac_group_1.csv"
data_df = pd.read_csv(filepath_or_buffer = csv_path, converters={"Embedding": lambda x: [float(num) for num in x.strip('[]').split()]})

# Lijsten maken van data
X = data_df['Embedding'].to_list()
y = data_df['Label'].to_list()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

## Logistic Regression

In [ ]:
# Model Trainen
LR = LogisticRegression()
LR.fit(X_train,y_train)

# Testen of model werkt
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))
print("Logistic Regression F1:",metrics.f1_score(y_test, predicted))
print('')

## Random Forest

In [ ]:
RF = RandomForestClassifier()
RF.fit(X_train, y_train)

# Testen of model werkt
predicted = RF.predict(X_test)
print("Random Forest Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Random Forest Precision:",metrics.precision_score(y_test, predicted))
print("Random Forest Recall:",metrics.recall_score(y_test, predicted))
print("Random Forest F1:",metrics.f1_score(y_test, predicted))
print('')

## Gradient Boosting

In [ ]:
GB = GradientBoostingClassifier()
GB.fit(X_train, y_train)

# Testen of model werkt
predicted = GB.predict(X_test)
print("Gradient Boosting Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Gradient Boosting Precision:",metrics.precision_score(y_test, predicted))
print("Gradient Boosting Recall:",metrics.recall_score(y_test, predicted))
print("Gradient Boosting F1:",metrics.f1_score(y_test, predicted))

## SVM

In [ ]:
param_grids = {
    'linear':{'C': [0.1, 1, 10],
              'kernel': ['linear'],
              'class_weight': [None, 'balanced']},
    'rbf':{'C': [0.1, 1, 10],
              'kernel': ['rbf'],
              'gamma': [0.1, 1, 'auto'],
              'class_weight': [None, 'balanced']},
    'poly':{'C': [0.1, 1, 10],
              'kernel': ['poly'],
              'gamma': [0.1, 1, 'auto'],
              'degree': [2, 3, 4],
              'class_weight': [None, 'balanced']}
}

best_f1_scores = {}
best_models = {}

# Iterate through each kernel type in param_grids
for kernel, params in param_grids.items():
    # Create the SVM model
    svm = SVC()

    # Perform grid search
    grid_search = GridSearchCV(estimator=svm, param_grid=params, cv=5, scoring='f1')
    grid_search.fit(X_train, y_train)

    # Get the best F1 score and best model for each kernel type
    best_f1_scores[kernel] = grid_search.best_score_
    best_models[kernel] = grid_search.best_estimator_
    print(f"Finished {kernel}")

# Find the kernel type with the highest F1 score
best_kernel = max(best_f1_scores, key=best_f1_scores.get)
best_f1_for_best_kernel = best_f1_scores[best_kernel]
best_model_for_best_kernel = best_models[best_kernel]

print(f"Best kernel type: {best_kernel}")
print(f"Best F1 score for the best kernel type: {best_f1_for_best_kernel:.4f}")
print(f"Best model parameters: {best_model_for_best_kernel.get_params()}")

# Testen of model werkt
predicted = best_model_for_best_kernel.predict(X_test)
print(predicted)
print(y_test)
print("Support Vector Machine Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Support Vector Machine Precision:",metrics.precision_score(y_test, predicted))
print("Support Vector Machine Recall:",metrics.recall_score(y_test, predicted))
print("Support Vector Machine F1:",metrics.f1_score(y_test, predicted))
print('')

Finished linear
Finished rbf
